# Elasticsearch Vector Store

Elasticsearch is a distributed, RESTful search and analytics engine, capable of performing both vector and keyword search. It is built on top of the Apache Lucene library.

[Signup](https://cloud.elastic.co/registration?utm_source=llama-index&utm_content=documentation) for a free trial.

Requires Elasticsearch 8.9.0 or higher.

In [1]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

## Running and connecting to Elasticsearch
Two ways to setup an Elasticsearch instance for use with:

### Elastic Cloud
Elastic Cloud is a managed Elasticsearch service. [Signup](https://cloud.elastic.co/registration?utm_source=llama-index&utm_content=documentation) for a free trial.

### Locally
Get started with Elasticsearch by running it locally. The easiest way is to use the official Elasticsearch Docker image. See the Elasticsearch Docker documentation for more information.

```bash
docker run -p 9200:9200 \
  -e "discovery.type=single-node" \
  -e "xpack.security.enabled=false" \
  -e "xpack.security.http.ssl.enabled=false" \
  -e "xpack.license.self_generated.type=trial" \
  docker.elastic.co/elasticsearch/elasticsearch:8.9.0
```

## Configuring ElasticsearchStore
The ElasticsearchStore class is used to connect to an Elasticsearch instance. It requires the following parameters:

        - index_name: Name of the Elasticsearch index. Required.
        - es_client: Optional. Pre-existing Elasticsearch client.
        - es_url: Optional. Elasticsearch URL.
        - es_cloud_id: Optional. Elasticsearch cloud ID.
        - es_api_key: Optional. Elasticsearch API key.
        - es_user: Optional. Elasticsearch username.
        - es_password: Optional. Elasticsearch password.
        - text_field: Optional. Name of the Elasticsearch field that stores the text.
        - vector_field: Optional. Name of the Elasticsearch field that stores the
                    embedding.
        - batch_size: Optional. Batch size for bulk indexing. Defaults to 200.
        - distance_strategy: Optional. Distance strategy to use for similarity search.
                        Defaults to "COSINE".

### Example: Connecting locally
```python
from llama_index.vector_stores import ElasticsearchStore

es = ElasticsearchStore(
    index_name="my_index",
    es_url="http://localhost:9200",
)
```

### Example: Connecting to Elastic Cloud with username and password

```python
from llama_index.vector_stores import ElasticsearchStore

es = ElasticsearchStore(
    index_name="my_index",
    es_cloud_id="<cloud-id>", # found within the deployment page
    es_user="elastic"
    es_password="<password>" # provided when creating deployment. Alternatively can reset password.
)
```

### Example: Connecting to Elastic Cloud with API Key

```python
from llama_index.vector_stores import ElasticsearchStore

es = ElasticsearchStore(
    index_name="my_index",
    es_cloud_id="<cloud-id>", # found within the deployment page
    es_api_key="<api-key>" # Create an API key within Kibana (Security -> API Keys)
)
```


#### Load documents, build VectorStoreIndex with Elasticsearch

In [2]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import ElasticsearchStore

INFO:numexpr.utils:Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


In [3]:
# load documents
documents = SimpleDirectoryReader("../data/paul_graham").load_data()

In [5]:
# initialize without metadata filter
from llama_index.storage.storage_context import StorageContext

vector_store = ElasticsearchStore(
    es_url="http://localhost:9200", 
    # Or with Elastic Cloud
    # es_cloud_id="my_cloud_id",
    # es_user="elastic",
    # es_password="my_password",
    index_name="paul_graham"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

INFO:elastic_transport.transport:GET http://localhost:9200/ [status:200 duration:0.024s]
GET http://localhost:9200/ [status:200 duration:0.024s]
INFO:elastic_transport.transport:HEAD http://localhost:9200/paul_graham [status:200 duration:0.011s]
HEAD http://localhost:9200/paul_graham [status:200 duration:0.011s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk?refresh=true [status:200 duration:0.115s]
PUT http://localhost:9200/_bulk?refresh=true [status:200 duration:0.115s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk?refresh=true [status:200 duration:0.083s]
PUT http://localhost:9200/_bulk?refresh=true [status:200 duration:0.083s]


## Basic Example
We are going to ask the query engine a question about the data we just indexed.

In [8]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("what were his investments in Y Combinator?")
print(response)

INFO:elastic_transport.transport:POST http://localhost:9200/paul_graham/_search [status:200 duration:0.030s]
POST http://localhost:9200/paul_graham/_search [status:200 duration:0.030s]
He invested $6k per founder, which in the typical two-founder case was $12k, in return for 6%.


## Metadata Filters
Here we are going to index a few documents with metadata so that we can apply filters to the query engine.

In [16]:
from llama_index.schema import TextNode

nodes = [
    TextNode(
        text="The Shawshank Redemption",
        metadata={
            "author": "Stephen King",
            "theme": "Friendship",
        },
    ),
    TextNode(
        text="The Godfather",
        metadata={
            "director": "Francis Ford Coppola",
            "theme": "Mafia",
        },
    ),
    TextNode(
        text="Inception",
        metadata={
            "director": "Christopher Nolan",
        },
    ),
]

# initialize the vector store
vector_store_metadata_example = ElasticsearchStore(
    index_name="movies_metadata_example",
    es_url="http://localhost:9200",
)
storage_context = StorageContext.from_defaults(vector_store=vector_store_metadata_example)
index = VectorStoreIndex(nodes, storage_context=storage_context)


# Metadata filter
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters

filters = MetadataFilters(filters=[ExactMatchFilter(key="theme", value="Mafia")])

retriever = index.as_retriever(filters=filters)

retriever.retrieve("What is inception about?")

INFO:elastic_transport.transport:GET http://localhost:9200/ [status:200 duration:0.012s]
GET http://localhost:9200/ [status:200 duration:0.012s]


INFO:elastic_transport.transport:HEAD http://localhost:9200/movies_metadata_example [status:404 duration:0.022s]
HEAD http://localhost:9200/movies_metadata_example [status:404 duration:0.022s]
INFO:elastic_transport.transport:PUT http://localhost:9200/movies_metadata_example [status:200 duration:0.099s]
PUT http://localhost:9200/movies_metadata_example [status:200 duration:0.099s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk?refresh=true [status:200 duration:0.053s]
PUT http://localhost:9200/_bulk?refresh=true [status:200 duration:0.053s]
INFO:elastic_transport.transport:PUT http://localhost:9200/_bulk?refresh=true [status:200 duration:0.023s]
PUT http://localhost:9200/_bulk?refresh=true [status:200 duration:0.023s]
INFO:elastic_transport.transport:POST http://localhost:9200/movies_metadata_example/_search [status:200 duration:0.034s]
POST http://localhost:9200/movies_metadata_example/_search [status:200 duration:0.034s]


[NodeWithScore(node=TextNode(id_='3b47c6b6-f01b-44fe-8f88-2249aad2a615', embedding=None, metadata={'director': 'Francis Ford Coppola', 'theme': 'Mafia'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='81cf4b9e847ba42e83fc401e31af8e17d629f0d5cf9c0c320ec7ac69dd0257e1', text='The Godfather', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.88513875)]

## Custom Filters and overriding Query 
llama-index supports ExactMatchFilters only at the moment. Elasticsearch supports a wide range of filters, including range filters, geo filters, and more. To use these filters, you can pass them in as a list of dictionaries to the `es_filter` parameter.

In [9]:
def custom_query(query, query_str):
    print("custom query", query)
    return query


query_engine = index.as_query_engine(
    vector_store_kwargs={
        "es_filter": [{"match": {"content": "growing up"}}],
        "custom_query": custom_query,
    }
)
response = query_engine.query("what were his investments in Y Combinator?")
print(response)

custom query {'knn': {'filter': [{'match': {'content': 'growing up'}}], 'field': 'embedding', 'query_vector': [0.002520269714295864, -0.03282919153571129, 0.016138022765517235, -0.029537975788116455, -0.006744919344782829, 0.01626248098909855, -0.03703309968113899, 0.002381983445957303, -0.003031929489225149, -0.003616189584136009, 0.032746221870183945, 0.030201751738786697, 0.011726687662303448, 0.005043996497988701, 0.0030665011145174503, 0.016207166016101837, 0.018115518614649773, -0.008539185859262943, 0.020825933665037155, -0.011595315299928188, -0.027754081413149834, -0.004622223321348429, -0.004750138148665428, -0.015363619662821293, -0.006496003828942776, 0.012860636226832867, 0.02331508882343769, -0.009368903934955597, -0.002686213469132781, 0.0029818005859851837, 0.032441992312669754, 0.0015107790241017938, -0.0023059258237481117, 0.02384057641029358, -0.029233746230602264, 0.003574703587219119, 0.0048296526074409485, 0.019401581957936287, 0.01830912008881569, -0.009375818073